In [1]:
import pandapower as pp #import pandapower
import pandas as pd
import math
from pandapower.plotting.plotly import simple_plotly
net = pp.create_empty_network() #create an empty network

In [2]:
# Loading the node csv

nodes_orig = pd.read_csv('network_data/nodes_with_im.csv')
nodes_orig['NodeID'] = nodes_orig['NodeID'].astype(str) 
spotload = nodes_orig[nodes_orig['NodeID'].str.contains('SPOTLOAD')].reset_index()
sub_con_nodes = pd.read_csv('network_data/sub_con_nodes.csv')
sub_con_nodes['NodeID'] = sub_con_nodes['NodeID'].astype(str) 

#intermediate_nodes = pd.read_csv('network_data/intermediate_nodes_sep.csv')
#intermediate_nodes['NodeID'] = intermediate_nodes['NodeID'].astype(str) 

subnet_nodes = pd.read_csv('network_data/subnet_nodes.csv')
subnet_nodes['NodeID'] = subnet_nodes['NodeID'].astype(str) 

spotload['NodeID'] = spotload['NodeID'].map(lambda x: x.rstrip('-SPOTLOAD')) 

with pd.option_context('display.max_rows', None, 'display.max_columns', 6):
    print(nodes_orig)

                                 NodeID         CoordX         CoordY
0                     42784779-SPOTLOAD  553936.808426  146115.680388
1                              42805981  554167.347000  145823.926000
2                     42762171-SPOTLOAD  554234.887258  145830.116397
3                              42773422  554224.880000  146486.816000
4                     42774787-SPOTLOAD  554004.786862  145857.488915
5                              42800598  554172.665000  145749.726000
6                              42826835  554213.574000  146132.659000
7                              42684822  554202.296000  145812.842000
8                              42692306  553907.670000  146018.676000
9                              42793458  554020.032000  145979.088000
10                             42677448  553987.234000  146063.703000
11                            160381938  553708.694000  145882.643000
12                             42689099  554258.868000  145618.281000
13                  

In [3]:
# Loading the section csv for the lines

section = pd.read_csv('network_data/result_section_better.csv', header=0)
cable = pd.read_csv('network_data/cable_combine.csv', header=0)

section = section[['SectionID', 'FromNodeID', 'ToNodeID']]
section['FromNodeID'] = section['FromNodeID'].astype(int)
#section[['ToNodeID']] = section[['ToNodeID']].astype(int)
section = section.applymap(str)
cable['SectionID'] = cable['SectionID'].astype(str)

cable_section = section.merge(cable, on='SectionID')
cable_section['B1'] = cable_section['B1'].astype(float)
cable_section['Length'] = cable_section['Length'].astype(float)
#cable_section['R1'] = cable_section['R1'].astype(float)

with pd.option_context('display.max_rows', None, 'display.max_columns', 6):
    print(cable_section)


      SectionID FromNodeID   ToNodeID   ...   Flags  Comments  Length
0      42571222   42901266   42901267   ...       0       NaN     0.0
1      42533575   42890834   42890827   ...       0       NaN     0.0
2      43939209   42787380   42687308   ...       0       NaN     7.0
3      43825506   42698357   42684664   ...       0       NaN    18.0
4      43971553   42703621   42694044   ...       0       NaN   238.0
5      94588448   52857530   94588294   ...       0       NaN    34.0
6      43895417   52889602   42697592   ...       0       NaN     3.0
7      42533570   42890835   42890827   ...       0       NaN     0.0
8      43906026   42785340   42692978   ...       0       NaN     4.0
9      43939204   42660293   42793224   ...       0       NaN    11.0
10     93849829   93848443   42890827   ...       0       NaN     0.0
11     43884803   42686073   42698393   ...       0       NaN    30.0
12     42571224   42901268   42901266   ...       0       NaN     0.0
13     52107522   42

In [4]:
cable_section_2 = cable_section.copy()

cable_section_2['X1'] = cable_section['X1'].replace(0.0000, 0.001)

cable_section

,SectionID,FromNodeID,ToNodeID,ID,R1,R0,X1,X0,B1,B0,...,OverallDiameter,ConcentricNeutralBeforeSheath,UserDefinedImpedances,Frequency,Temperature,ImpedancesNote,Favorite,Flags,Comments,Length
0,42571222,42901266,42901267,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,0.0
1,42533575,42890834,42890827,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,0.0
2,43939209,42787380,42687308,B-GKN-K-16,1.4662,0.00,0.0900,0.000,137.500,0.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,7.0
3,43825506,42698357,42684664,B-P-50+25,0.4476,0.00,0.0900,0.000,282.740,0.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,18.0
4,43971553,42703621,42694044,B-XN-150-AL,0.2633,0.00,0.0780,0.000,230.360,0.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,238.0
5,94588448,52857530,94588294,B-GKN-K-150 AL+95,0.2450,1.00,0.0710,1.000,1.000,1.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,34.0
6,43895417,52889602,42697592,B-T-95,0.2238,0.00,0.0820,0.000,706.680,0.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,3.0
7,42533570,42890835,42890827,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,0.0
8,43906026,42785340,42692978,B-P-25+16,0.8409,0.00,0.0940,0.000,259.180,0.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,4.0
9,43939204,42660293,42793224,B-P-5,4.7100,0.00,0.1200,0.000,123.700,0.000,...,28.0,0,1,-1.0,-274.0,NaN,0,0,NaN,11.0


In [5]:
## Creating rated voltage of the nodes

cable_section_2['RX'] = cable_section_2['R1'] / cable_section_2['X1']
r_voltage = []

for row in cable_section_2['RX']:
    #if row
     #   r_voltage.append(21.0)
    if 0 <= row  <= 1:
        r_voltage.append(10.6) 
    if row > 1:
        r_voltage.append(0.42)
     
        
cable_section_2['r_voltage'] = r_voltage

cable_section_3 = cable_section_2[['FromNodeID','r_voltage']].copy()
cable_section_4 = cable_section_2[['ToNodeID','r_voltage']].copy()


cable_section_3 = cable_section_3.rename(index=str, columns={'FromNodeID':'NodeID'})
cable_section_4 = cable_section_4.rename(index=str, columns={'ToNodeID':'NodeID'})

cable_section_5 = cable_section_3.append(cable_section_4, ignore_index=True)

cable_section_5 = cable_section_5.drop_duplicates(subset='NodeID', keep='first', inplace=False)

nodes = cable_section_5.merge(nodes_orig, on='NodeID')

sub_con_nodes = cable_section_5.merge(sub_con_nodes, on='NodeID')
subnet_nodes = cable_section_5.merge(subnet_nodes, on='NodeID')

frames = [nodes,sub_con_nodes,subnet_nodes] 
nodes_complet = pd.concat(frames)

nodes_complet = nodes_complet.drop_duplicates(subset='NodeID', keep='first', inplace=False)

nodes_complet = nodes_complet.reset_index()
spotload['NodeID'] = spotload['NodeID'].map(lambda x: x.rstrip('-SPOTLOAD')) 

#spotload_check = []
spotload_check = spotload[~spotload['NodeID'].isin(nodes_complet['NodeID'])]
 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(nodes_complet)


      index     NodeID  r_voltage         CoordX         CoordY
0         0   42787380       0.42  554020.475000  146396.927000
1         1   42698357       0.42  554007.887000  146422.604000
2         2   42703621       0.42  554016.160000  146347.766000
3         3   42785340       0.42  554051.114000  146388.604000
4         4   42660293       0.42  554018.517000  146409.068000
5         5   42686073       0.42  554025.107000  146394.201000
6         6   42699677       0.42  553987.699000  146441.836000
7         7   94588294       0.42  554092.565000  146310.190000
8         8   42767154       0.42  554031.122000  146433.006000
9         9   42693414       0.42  554038.477000  146383.651000
10       10   42691844       0.42  553974.454000  146430.939000
11       11   42783662       0.42  554027.075000  146388.316000
12       12   42700105       0.42  554031.871000  146392.063000
13       13   42822267       0.42  554001.929000  145914.670000
14       14   42826833       0.42  55422

In [6]:
### Note ###
##########################################################################################################################
### Defining a line with length zero leads to a division by zero in the power flow                                     ###
### and is therefore not allowed. Lines with a very low impedance might lead to convergence problems                   ###
### in the power flow for the same reason.                                                                             ###
### If you want to directly connect two buses, please use the switch element instead of a line with a small impedance! ###
##########################################################################################################################

In [7]:
cable_section_low_imp = cable_section_2[cable_section_2['X1'] < 0.005]
cable_section_zero_length =  cable_section_2[cable_section_2['Length'] < 0.5]


cable_section_2 = cable_section_2[cable_section_2['X1'] >= 0.005]
cable_section_2 = cable_section_2[cable_section_2['Length'] >= 0.5]

cable_section_zero_length

,SectionID,FromNodeID,ToNodeID,ID,R1,R0,X1,X0,B1,B0,...,UserDefinedImpedances,Frequency,Temperature,ImpedancesNote,Favorite,Flags,Comments,Length,RX,r_voltage
0,42571222,42901266,42901267,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
1,42533575,42890834,42890827,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
7,42533570,42890835,42890827,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
10,93849829,93848443,42890827,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
12,42571224,42901268,42901266,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
16,42571223,42909247,42901266,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
19,42533571,42890831,42890827,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
22,42571225,42909248,42901266,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
34,93849824,42890827,93848381,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42
40,42533572,42890829,42890827,B-FIC,0.4158,1.00,0.3269,1.000,3.544,1.000,...,1,-1.0,-274.0,NaN,0,0,NaN,0.0,1.271949,0.42


In [8]:
# Nodes/busses

for i, row in nodes_complet.iterrows():
    pp.create_bus(net, name=row.NodeID, vn_kv=row.r_voltage, geodata = (row.CoordX,row.CoordY))

# Subnetwork connection nodes 
#for i, row in sub_con_nodes.iterrows():
#    pp.create_bus(net, name=sub_con_nodes['NodeID'][i], vn_kv=row.r_voltage, geodata = (row.CoordX,row.CoordY))

# Intermediate nodes
#for i, row in intermediate_nodes.iterrows():
#    pp.create_bus(net, name=row.NodeID, vn_kv=112, type='n')

# subnetwork "nodes"
#for i, row in subnet_nodes.iterrows():
#    pp.create_bus(net, name=row.NodeID, vn_kv=row.r_voltage, geodata = (row.CoordX,row.CoordY))


In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 6):
    print(net.bus) # show bus table
#net.bus
#net.bus_geodata

           name  vn_kv type  zone  in_service
0      42787380   0.42    b  None        True
1      42698357   0.42    b  None        True
2      42703621   0.42    b  None        True
3      42785340   0.42    b  None        True
4      42660293   0.42    b  None        True
5      42686073   0.42    b  None        True
6      42699677   0.42    b  None        True
7      94588294   0.42    b  None        True
8      42767154   0.42    b  None        True
9      42693414   0.42    b  None        True
10     42691844   0.42    b  None        True
11     42783662   0.42    b  None        True
12     42700105   0.42    b  None        True
13     42822267   0.42    b  None        True
14     42826833   0.42    b  None        True
15    137006430   0.42    b  None        True
16     42706732   0.42    b  None        True
17     42826835   0.42    b  None        True
18     42827073   0.42    b  None        True
19     42858229   0.42    b  None        True
20     88877489   0.42    b  None 

In [10]:
# creating the lines
# Underground
for _, hv_line in cable_section_2.iterrows():
        from_bus = pp.get_element_index(net, "bus", hv_line.FromNodeID)
        to_bus = pp.get_element_index(net, "bus", hv_line.ToNodeID)
        pp.create_line_from_parameters(net, from_bus, to_bus, length_km=hv_line.Length/1000,
                                       r_ohm_per_km = hv_line.R1, x_ohm_per_km = hv_line.X1, 
                                       c_nf_per_km = hv_line.B1 *math.pi*2*50/1000 ,
                                       max_i_ka = hv_line.Amps_4/1000, name=hv_line.SectionID , parallel=7,
                                       type = "cs", df = 1)
 
# show line table
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,max_i_ka,df,parallel,type,in_service
0,43939209,None,0,773,0.007,1.4662,0.0900,43.196899,0.110,1.0,7,cs,True
1,43825506,None,1,774,0.018,0.4476,0.0900,88.825391,0.150,1.0,7,cs,True
2,43971553,None,2,775,0.238,0.2633,0.0780,72.369728,0.319,1.0,7,cs,True
3,94588448,None,1343,7,0.034,0.2450,0.0710,0.314159,0.281,1.0,7,cs,True
4,43895417,None,2040,776,0.003,0.2238,0.0820,222.010070,0.255,1.0,7,cs,True
5,43906026,None,3,777,0.004,0.8409,0.0940,81.423798,0.105,1.0,7,cs,True
6,43939204,None,4,778,0.011,4.7100,0.1200,38.861501,0.050,1.0,7,cs,True
7,43884803,None,5,779,0.030,0.2238,0.0860,111.005035,0.220,1.0,7,cs,True
8,52107522,None,2044,2052,0.001,0.4158,0.3269,1.113380,0.305,1.0,7,cs,True
9,52101384,None,2045,1344,0.001,0.4158,0.3269,1.113380,0.305,1.0,7,cs,True


In [11]:
# Transformer 

transformer = pd.read_csv('network_data/transformer_combined.csv')

transformer = transformer[["SectionID" , "FromNodeID", "DeviceNumber", "KVA", "KVLLprim", "KVLLsec"]].copy()

transformer[["SectionID" , "FromNodeID", "DeviceNumber"]] = transformer[["SectionID" , "FromNodeID", "DeviceNumber"]].astype(int)
transformer[["SectionID" , "FromNodeID", "DeviceNumber"]] = transformer[["SectionID" , "FromNodeID", "DeviceNumber"]].astype(str)

transformer

,SectionID,FromNodeID,DeviceNumber,KVA,KVLLprim,KVLLsec
0,42547360,42934976,42934975,400.0,10.6,0.42
1,42571987,42905802,42905802,630.0,10.6,0.42
2,42590298,42980182,42980182,400.0,10.6,0.42
3,42552624,42878165,42882930,100.0,10.6,0.42
4,42574167,42902140,42902140,630.0,21.0,0.42
5,214551743,105470910,105470910,630.0,21.0,0.42
6,42595956,42941227,42882789,10000.0,21.0,10.60
7,42613835,42951838,42951839,630.0,21.0,0.42
8,42571862,42891069,42891068,630.0,10.6,0.42
9,42574494,42945679,42945679,630.0,21.0,0.42


In [12]:
# Low Voltage Transforer


transformer_mid_low_voltage = transformer[transformer['KVLLsec'] == 0.42]

transformer_10_to_04 = transformer_mid_low_voltage[transformer_mid_low_voltage['KVLLprim'] == 10.6]
transformer_10_to_04_100 = transformer_10_to_04[transformer_10_to_04['KVA'] == 100].reset_index()
transformer_10_to_04_400 = transformer_10_to_04[transformer_10_to_04['KVA'] == 400].reset_index()
transformer_10_to_04_630 = transformer_10_to_04[transformer_10_to_04['KVA'] == 630].reset_index()
transformer_10_to_04_1000 = transformer_10_to_04[transformer_10_to_04['KVA'] == 1000].reset_index()


transformer_21_to_04 = transformer_mid_low_voltage[transformer_mid_low_voltage['KVLLprim'] == 21.0]

transformer_21_to_04_400 = transformer_21_to_04[transformer_21_to_04['KVA'] == 400].reset_index()
transformer_21_to_04_630 = transformer_21_to_04[transformer_21_to_04['KVA'] == 630].reset_index()
transformer_21_to_04_1000 = transformer_21_to_04[transformer_21_to_04['KVA'] == 1000].reset_index()

transformer_mid_mid_voltage = transformer[transformer['KVLLsec'] == 10.6]
transformer_21_to_10_10000 = transformer_mid_mid_voltage[transformer_mid_mid_voltage['KVLLprim'] == 21.0].reset_index()

for _,row in transformer_10_to_04_100.iterrows():
    hv_bus = pp.get_element_index(net, "bus", row.FromNodeID)
    lv_bus = pp.get_element_index(net, "bus", row.DeviceNumber)
    pp.create_transformer_from_parameters(net, hv_bus, lv_bus, sn_kva=100,
                                        vn_hv_kv=10.6, vn_lv_kv=0.42, vscr_percent=1.325, vsc_percent=4,
                                        pfe_kw=0.95, i0_percent=0.2375, tp_side="hv", tp_mid=0, tp_min=-2,
                                        tp_max=2, tp_st_percent=2.5, tp_pos=0, shift_degree=150, name='Trafo_'+ row.SectionID)

for _,row in transformer_10_to_04_400.iterrows():
    hv_bus = pp.get_element_index(net, "bus", row.FromNodeID)
    lv_bus = pp.get_element_index(net, "bus", row.DeviceNumber)
    pp.create_transformer(net, hv_bus, lv_bus, std_type='0.4 MVA 10/0.4 kV', name='Trafo_'+ row.SectionID,
                          in_service=True)

for _,row in transformer_10_to_04_630.iterrows():
    hv_bus = pp.get_element_index(net, "bus", row.FromNodeID)
    lv_bus = pp.get_element_index(net, "bus", row.DeviceNumber)
    pp.create_transformer(net, hv_bus, lv_bus, std_type='0.63 MVA 10/0.4 kV', name='Trafo_'+ row.SectionID,
                          in_service=True) 

for _,row in transformer_10_to_04_1000.iterrows():
    hv_bus = pp.get_element_index(net, "bus", row.FromNodeID)
    lv_bus = pp.get_element_index(net, "bus", row.DeviceNumber)
    pp.create_transformer_from_parameters(net, hv_bus, lv_bus, sn_kva=400,
                                      vn_hv_kv=10.6, vn_lv_kv=0.42, vscr_percent=1.325, vsc_percent=4,
                                      pfe_kw=0.95, i0_percent=0.2375, tp_side="hv", tp_mid=0, tp_min=-2,
                                      tp_max=2, tp_st_percent=2.5, tp_pos=0, shift_degree=150, name='Trafo_'+ row.SectionID)


for _,row in transformer_21_to_04_400.iterrows():
    hv_bus = pp.get_element_index(net, "bus", row.FromNodeID)
    lv_bus = pp.get_element_index(net, "bus", row.DeviceNumber)
    pp.create_transformer(net, hv_bus, lv_bus, std_type='0.4 MVA 20/0.4 kV', name='Trafo_'+ row.SectionID,
                          in_service=True)

for _,row in transformer_21_to_04_630.iterrows():
    hv_bus = pp.get_element_index(net, "bus", row.FromNodeID)
    lv_bus = pp.get_element_index(net, "bus", row.DeviceNumber)
    pp.create_transformer(net, hv_bus, lv_bus, std_type='0.63 MVA 20/0.4 kV', name='Trafo_'+ row.SectionID, 
                          in_service=True)
    
for _,row in transformer_21_to_04_1000.iterrows():
    hv_bus = pp.get_element_index(net, "bus", row.FromNodeID)
    lv_bus = pp.get_element_index(net, "bus", row.DeviceNumber)
    pp.create_transformer_from_parameters(net, hv_bus, lv_bus, sn_kva=400,
                                      vn_hv_kv=10.6, vn_lv_kv=0.42, vscr_percent=1.325, vsc_percent=4,
                                      pfe_kw=0.95, i0_percent=0.2375, tp_side="hv", tp_mid=0, tp_min=-2,
                                      tp_max=2, tp_st_percent=2.5, tp_pos=0, shift_degree=150, name='Trafo_'+ row.SectionID)
    
    
for _,row in transformer_21_to_10_10000.iterrows():
    hv_bus = pp.get_element_index(net, "bus", row.FromNodeID)
    lv_bus = pp.get_element_index(net, "bus", row.DeviceNumber)
    pp.create_transformer_from_parameters(net, hv_bus, lv_bus, sn_kva=400,
                                      vn_hv_kv=10.6, vn_lv_kv=0.42, vscr_percent=1.325, vsc_percent=4,
                                      pfe_kw=0.95, i0_percent=0.2375, tp_side="hv", tp_mid=0, tp_min=-2,
                                      tp_max=2, tp_st_percent=2.5, tp_pos=0, shift_degree=150, name='Trafo_'+ row.SectionID)
    
    





In [13]:
# Transformer not in the network 
trans_check = transformer[~transformer['SectionID'].isin(cable_section['SectionID'])]
trans_check

### Overview of the transformer ###
#transformer_10_to_04_100 # 1 Transformer, which is not in the network
#transformer_10_to_04_400 # 6 Transformers: included
#transformer_10_to_04_630 # 5 Transformers: included
#transformer_10_to_04_1000 # 1 Transformer, where the parameters are missing

#transformer_21_to_04_400 # 3 transformer: included
#transformer_21_to_04_630 # 15 transformer: included

#transformer_21_to_04_1000 # 4 transformers, where the parameters are missing

#transformer_21_to_10_10000 # 3 transformers which are all not in the network

,SectionID,FromNodeID,DeviceNumber,KVA,KVLLprim,KVLLsec


In [14]:
net.trafo # show trafo table
#show only low voltage transformer


,name,std_type,hv_bus,lv_bus,sn_kva,vn_hv_kv,vn_lv_kv,vsc_percent,vscr_percent,pfe_kw,...,tp_side,tp_mid,tp_min,tp_max,tp_st_percent,tp_st_degree,tp_pos,parallel,df,in_service
0,Trafo_42552624,None,3181,3150,100.0,10.6,0.42,4.0,1.3250,0.95,...,hv,0,-2,2,2.5,NaN,0,1,1.0,True
1,Trafo_42547360,0.4 MVA 10/0.4 kV,2607,2576,400.0,10.0,0.40,4.0,1.3250,0.95,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True
2,Trafo_42590298,0.4 MVA 10/0.4 kV,2634,2634,400.0,10.0,0.40,4.0,1.3250,0.95,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True
3,Trafo_42569691,0.4 MVA 10/0.4 kV,2476,2476,400.0,10.0,0.40,4.0,1.3250,0.95,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True
4,Trafo_214659777,0.4 MVA 10/0.4 kV,2609,3330,400.0,10.0,0.40,4.0,1.3250,0.95,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True
5,Trafo_42570236,0.4 MVA 10/0.4 kV,2621,2621,400.0,10.0,0.40,4.0,1.3250,0.95,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True
6,Trafo_42576786,0.4 MVA 10/0.4 kV,2442,2442,400.0,10.0,0.40,4.0,1.3250,0.95,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True
7,Trafo_42571987,0.63 MVA 10/0.4 kV,2625,2625,630.0,10.0,0.40,4.0,1.0794,1.18,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True
8,Trafo_42571862,0.63 MVA 10/0.4 kV,2585,2618,630.0,10.0,0.40,4.0,1.0794,1.18,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True
9,Trafo_42554475,0.63 MVA 10/0.4 kV,2583,3328,630.0,10.0,0.40,4.0,1.0794,1.18,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True


In [15]:
# Switches, no network infos

#hv_bus_sw = pd.read_csv('network_data/switch_equipment.csv', header=0, decimal=',')

#hv_bus_sw

In [16]:
# Bus-bus switches

#for _, switch in hv_bus_sw.iterrows():
#    from_bus = pp.get_element_index(net, "bus", switch.from_bus)
#    to_bus = pp.get_element_index(net, "bus", switch.to_bus)
#    pp.create_switch(net, from_bus, to_bus, et=switch.et, closed=switch.closed, type=switch.type, name=switch.bus_name)



In [17]:
# Bus-line switches
#hv_buses = net.bus[(net.bus.vn_kv == 380) | (net.bus.vn_kv == 110)].index
#hv_ls = net.line[(net.line.from_bus.isin(hv_buses)) & (net.line.to_bus.isin(hv_buses))]
#for _, line in hv_ls.iterrows():
#        pp.create_switch(net, line.from_bus, line.name, et='l', closed=True, type='LBS', name='Switch %s - %s' % (net.bus.name.at[line.from_bus], line['name']))
#        pp.create_switch(net, line.to_bus, line.name, et='l', closed=True, type='LBS', name='Switch %s - %s' % (net.bus.name.at[line.to_bus], line['name']))



In [18]:
# Trafo-line switches
#pp.create_switch(net, pp.get_element_index(net, "bus", 'Bus DB 2'), pp.get_element_index(net, "trafo", 'EHV-HV-Trafo'), et='t', closed=True, type='LBS', name='Switch DB2 - EHV-HV-Trafo')
#pp.create_switch(net, pp.get_element_index(net, "bus", 'Bus SB 1'), pp.get_element_index(net, "trafo", 'EHV-HV-Trafo'), et='t', closed=True, type='LBS', name='Switch SB1 - EHV-HV-Trafo')

# show switch table
#net.switch

In [19]:
# Load

hv_loads = pd.read_csv('network_data/spotload.csv', sep=',', header=0, decimal=',')
load_equivalent = pd.read_csv('network_data/load_equivalent.csv', sep=',', header=0, decimal=',')
custom_load = pd.read_csv('network_data/customer_load.csv', sep=',', header=0, decimal=',')

hv_loads = hv_loads[['SectionID','DeviceNumber','CustomerNumber','LoadModelID','ConnectedKVA', 'Value1','Value2', 'KWH']]

custom_load = custom_load[['ID',
 'ConstantPower',
 'ConstantCurrent',
 'ConstantImpedance',
 'UtilizationFactor',
 'PowerFactor',
 'ConstantImpedanceZQ',
 'ConstantCurrentIP',
 'ConstantCurrentIQ',
 'ConstantPowerPP',
 'ConstantPowerPQ',
 'ExponentialModelP',
 'ExponentialModelQ',
 'LoadFlowVoltagePercentOfNominal',
 'LossesPerCustomer',
 'ConnectedCenterTap1N',
 'ConnectedCenterTap2N',]]



custom_load
#load_equivalent
#hv_loads
#spotload

# show load table
#net.load

,ID,ConstantPower,ConstantCurrent,ConstantImpedance,UtilizationFactor,PowerFactor,ConstantImpedanceZQ,ConstantCurrentIP,ConstantCurrentIQ,ConstantPowerPP,ConstantPowerPQ,ExponentialModelP,ExponentialModelQ,LoadFlowVoltagePercentOfNominal,LossesPerCustomer,ConnectedCenterTap1N,ConnectedCenterTap2N
0,BT-DUP-F,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
1,BT-DUP-H,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
2,DOUBLE,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
3,EP-NORM,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
4,I-COURT,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
5,I-LONG,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
6,MT-DUP-F,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
7,MT-DUP-H,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
8,OTHER,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000
9,SIMPLE,100.000000,0.000000,0.000000,100.000000,95.000000,0.000000,0.000000,0.000000,100.000000,100.000000,0.000000,0.000000,80.000000,0.000000,50.000000,50.000000


In [20]:
for _, load in spotload.iterrows():
    bus_idx = pp.get_element_index(net, "bus", load.NodeID)
##    pp.create_load(net, bus_idx, p_kw=load.p, q_kvar=load.q, scaling=,in_service=True, name=load.load_name)
    pp.create_load_from_cosphi(net, bus_idx, sn_kva=100, cos_phi=0.95, mode="ind", name = load.NodeID)

net.load

,name,bus,p_kw,q_kvar,const_z_percent,const_i_percent,sn_kva,scaling,in_service,type
0,42784779,516,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
1,42762171,219,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
2,42774787,1329,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
3,42767560,986,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
4,42791070,351,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
5,42766857,67,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
6,42769631,412,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
7,42770999,470,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
8,42759255,1141,95.0,31.22499,0.0,0.0,100.0,1.0,True,None
9,42763745,1243,95.0,31.22499,0.0,0.0,100.0,1.0,True,None


In [21]:
# Static generators

pv_gen = pd.read_csv('network_data/pv_generator.csv', sep=',', header=0, decimal=',')
pv_nodes = nodes_orig[nodes_orig['NodeID'].str.contains('PHOTO')].reset_index()
  
pv_gen[["ActiveGeneration", "PowerFactor"]] = pv_gen[["ActiveGeneration", "PowerFactor"]].astype(float)

# Change the name of the pv device, so it fit with the node
pv_gen['DeviceNumber'] = pv_gen['DeviceNumber'].apply(lambda x: x.split('-')[0])
# only take the average value into consideration
pv_gen = pv_gen[pv_gen['LoadModelName'] == 'MOYENNE']

pv_nodes
pv_gen

,DeviceNumber,DeviceType,LoadModelName,ActiveGeneration,PowerFactor
2,42778481,45,MOYENNE,12.81238,100.0
5,42816058,45,MOYENNE,10.98204,100.0
8,42819953,45,MOYENNE,7.32136,100.0
11,42819954,45,MOYENNE,7.32136,100.0
14,78874605,45,MOYENNE,10.98204,100.0
17,42783662,45,MOYENNE,5.49102,100.0
20,42819954,45,MOYENNE,7.32136,100.0
23,42783436,45,MOYENNE,3.66068,100.0
26,42762511,45,MOYENNE,5.49102,100.0
29,42819953,45,MOYENNE,7.32136,100.0


In [22]:
# creating the pv table

for _,row in pv_gen.iterrows():
    phi = math.acos(row.PowerFactor/100)
    pp.create_sgen(net, pp.get_element_index(net, "bus", row.DeviceNumber), p_kw=-row.ActiveGeneration,
                   q_kvar=(-1)*math.tan(phi)*row.ActiveGeneration, type='PV', name=row.DeviceNumber)

# show static generator table
#pv_gen
net.sgen

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
0,42778481,1051,-12.81238,-0.0,NaN,1.0,True,PV
1,42816058,850,-10.98204,-0.0,NaN,1.0,True,PV
2,42819953,849,-7.32136,-0.0,NaN,1.0,True,PV
3,42819954,1259,-7.32136,-0.0,NaN,1.0,True,PV
4,78874605,762,-10.98204,-0.0,NaN,1.0,True,PV
5,42783662,11,-5.49102,-0.0,NaN,1.0,True,PV
6,42819954,1259,-7.32136,-0.0,NaN,1.0,True,PV
7,42783436,1085,-3.66068,-0.0,NaN,1.0,True,PV
8,42762511,1180,-5.49102,-0.0,NaN,1.0,True,PV
9,42819953,849,-7.32136,-0.0,NaN,1.0,True,PV


In [23]:
pp.create_ext_grid(net, pp.get_element_index(net, "bus", '42941227'), vm_pu=1, va_degree=0, name='External grid',
                   s_sc_max_mva=10000, rx_max=0.1, rx_min=0.1) 
# only FromNodeID for this bus, 
pp.create_ext_grid(net, pp.get_element_index(net, "bus", '42959671'), vm_pu=1, va_degree=0, name='External grid',
                   s_sc_max_mva=10000, rx_max=0.1, rx_min=0.1) 
# only ToNodeID
pp.create_ext_grid(net, pp.get_element_index(net, "bus", '42882790'), vm_pu=1, va_degree=0, name='External grid',
                   s_sc_max_mva=10000, rx_max=0.1, rx_min=0.1) 
# both from and ToNodeID, but with many following sections

# connected bus is just a guess: bus connected to 21 kV to 0.4 kV transformer with 10000 kVA, with ~default values

net.ext_grid # show external grid table

,name,bus,vm_pu,va_degree,in_service,s_sc_max_mva,rx_min,rx_max
0,External grid,3153,1.0,0.0,True,10000.0,0.1,0.1
1,External grid,3454,1.0,0.0,True,10000.0,0.1,0.1
2,External grid,3159,1.0,0.0,True,10000.0,0.1,0.1


In [24]:
pp.runpp(net, calculate_voltage_angles=True, init="dc")
net

This pandapower network includes the following parameter tables:
   - bus (3464 elements)
   - load (456 elements)
   - sgen (21 elements)
   - ext_grid (3 elements)
   - line (2233 elements)
   - trafo (38 elements)
   - bus_geodata (3464 elements)
 and the following results tables:
   - res_ext_grid (3 elements)
   - res_load (456 elements)
   - res_sgen (21 elements)
   - res_bus (3464 elements)
   - res_line (2233 elements)
   - res_trafo (38 elements)

In [25]:
net.res_ext_grid

,p_kw,q_kvar
0,-9.499817e-01,-0.000000e+00
1,-2.258190e+06,-6.431898e+06
2,-2.258190e+06,-6.431898e+06


In [26]:
net.res_load

,p_kw,q_kvar
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [27]:
net.res_load

,p_kw,q_kvar
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [28]:
net.res_sgen

,p_kw,q_kvar
0,-0.0,-0.0
1,-0.0,-0.0
2,-0.0,-0.0
3,-0.0,-0.0
4,-0.0,-0.0
5,-0.0,-0.0
6,-0.0,-0.0
7,-0.0,-0.0
8,-0.0,-0.0
9,-0.0,-0.0


In [29]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,NaN,NaN,0.000000e+00,0.000000e+00
1,NaN,NaN,0.000000e+00,0.000000e+00
2,NaN,NaN,0.000000e+00,0.000000e+00
3,NaN,NaN,0.000000e+00,0.000000e+00
4,NaN,NaN,0.000000e+00,0.000000e+00
5,NaN,NaN,0.000000e+00,0.000000e+00
6,NaN,NaN,0.000000e+00,0.000000e+00
7,NaN,NaN,0.000000e+00,0.000000e+00
8,NaN,NaN,0.000000e+00,0.000000e+00
9,NaN,NaN,0.000000e+00,0.000000e+00


In [30]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
6,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
7,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
8,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
9,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [31]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
1,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
2,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
3,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
4,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
5,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
6,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
7,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
8,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
9,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN


In [33]:
simple_plotly(net)
#pp.plotting.plotly.simple_plotly(net, use_line_geodata=False)

NameError: name 'Line' is not defined